In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(r'C:\Users\matte\OneDrive\Desktop\GitHub\data\causal\daily_restaurant_sales.csv')
df.head()

,rest_id,day,month,weekday,weekend,is_holiday,is_dec,is_nov,competitors_price,discounts,sales
0,0,2016-01-01,1,4,False,True,False,False,2.88,0,79.0
1,0,2016-01-02,1,5,True,False,False,False,2.64,0,57.0
2,0,2016-01-03,1,6,True,False,False,False,2.08,5,294.0
3,0,2016-01-04,1,0,False,False,False,False,3.37,15,676.5
4,0,2016-01-05,1,1,False,False,False,False,3.79,0,66.0


In [5]:
import statsmodels.formula.api as smf

X = ["C(month)", "C(weekday)", "is_holiday", "competitors_price"]
regr_cate = smf.ols(f"sales ~ discounts*({'+'.join(X)})",
                    data=df).fit()

In [6]:
ols_cate_pred = (
            regr_cate.predict(df.assign(discounts=df["discounts"]+1)) 
            -regr_cate.predict(df)
        )


In [7]:
train = df.query("day<'2018-01-01'")
test = df.query("day>='2018-01-01'")

In [8]:
data = df

In [9]:
X = ["C(month)", "C(weekday)", "is_holiday", "competitors_price"]
regr_model = smf.ols(f"sales ~ discounts*({'+'.join(X)})",
                             data=train).fit()

# prediction from test data
cate_pred = (
            regr_model.predict(test.assign(discounts=test["discounts"]+1)) 
            -regr_model.predict(test)
        )

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

X = ["month", "weekday", "is_holiday", "competitors_price", "discounts"]
y = "sales"

np.random.seed(1)
ml_model = GradientBoostingRegressor(n_estimators=50).fit(train[X],
                                                                  train[y])

ml_pred = ml_model.predict(test[X])

In [11]:
np.random.seed(123)

test_pred = test.assign(
            ml_pred=ml_pred,
            cate_pred=cate_pred,
            rand_m_pred=np.random.uniform(-1, 1, len(test)),
        )

In [12]:
from toolz import curry

# estimate the slope
@curry
def effect(data, y, t):
    return(np.sum((data[t]-data[t].mean())*data[y]) /
           np.sum((data[t]-data[t].mean())**2))

In [13]:
effect(test, 'sales', 'discounts')

32.16196368039615

In [14]:
def effect_by_quantile(df, pred, y, t, q=10):
             
             # makes quantile partitions
    groups = np.round(pd.IntervalIndex(pd.qcut(df[pred], q=q)).mid, 2) 
             
    return (df
                     .assign(**{f"{pred}_quantile": groups})
                     .groupby(f"{pred}_quantile")
                     # estimate the effect on each quantile
                     .apply(effect(y=y, t=t)))

         
effect_by_quantile(test_pred, "cate_pred", y="sales", t="discounts")

cate_pred_quantile
17.50    20.494153
23.93    24.782101
26.85    27.494156
28.95    28.833993
30.81    29.604257
32.68    32.216500
34.65    35.889459
36.75    36.846889
39.40    39.125449
47.36    44.272549
dtype: float64

In [15]:
test_pred

,rest_id,day,month,weekday,weekend,is_holiday,is_dec,is_nov,competitors_price,discounts,sales,ml_pred,cate_pred,rand_m_pred
731,0,2018-01-01,1,0,False,True,False,False,4.92,5,251.5,236.312960,41.355802,0.392938
732,0,2018-01-02,1,1,False,False,False,False,3.06,10,541.0,470.218050,44.743887,-0.427721
733,0,2018-01-03,1,2,False,False,False,False,4.61,10,431.0,429.180652,39.783798,-0.546297
734,0,2018-01-04,1,3,False,False,False,False,4.84,20,760.0,769.159322,40.770278,0.102630
735,0,2018-01-05,1,4,False,False,False,False,6.29,0,78.0,83.426070,40.666949,0.438938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7674,6,2018-12-28,12,4,False,False,True,False,6.14,15,771.0,598.981916,42.446960,-0.672354
7675,6,2018-12-29,12,5,True,False,True,False,9.29,5,229.0,236.103487,28.696779,0.581846
7676,6,2018-12-30,12,6,True,False,True,False,5.29,10,458.0,411.551171,38.631153,-0.333955
7677,6,2018-12-31,12,0,False,False,True,False,6.78,10,403.0,390.745578,34.023585,0.506215


In [23]:
def cumulative_effect_curve(dataset, prediction, y, t,
                                     ascending=False, steps=100):
    size = len(dataset)
    ordered_df = (dataset
                .sort_values(prediction, ascending=ascending)
                .reset_index(drop=True))
             
    steps = np.linspace(size/steps, size, steps).round(0)

    return np.array([effect(ordered_df.query(f'index<={row}'), t=t, y=y)
                            for row in steps])

cumulative_effect_curve(test_pred, "cate_pred", "sales", "discounts")

array([49.65116279, 49.37712454, 46.20360341, 45.9770684 , 45.31711812,
       45.23107566, 44.91075165, 44.74166167, 44.56309576, 44.27254859,
       43.95937553, 43.66601716, 43.30710192, 42.71556602, 42.57717619,
       42.14190207, 41.9120469 , 41.73012555, 41.59626585, 41.38073714,
       41.11778111, 41.04864418, 40.95933655, 40.6949357 , 40.5585599 ,
       40.32182723, 40.19431251, 40.02119492, 39.92014581, 39.58537488,
       39.44592589, 39.53382731, 39.41985355, 39.29287716, 39.27763551,
       39.14056914, 39.04402217, 38.98421346, 38.9022713 , 38.71601845,
       38.72423427, 38.53676256, 38.56709164, 38.36661748, 38.27685106,
       38.06110588, 37.99479087, 37.89750017, 37.72699021, 37.55580363,
       37.35825912, 37.28006651, 37.21139791, 37.06210857, 37.00753334,
       36.87259686, 36.74299254, 36.52014459, 36.4873419 , 36.38463167,
       36.21632077, 36.10908232, 35.90865968, 35.73915746, 35.56123101,
       35.42248759, 35.37393053, 35.25994908, 35.24501879, 35.14